In [1]:
using Turing
using LimberJack
using CSV
using NPZ
using FITSIO

In [2]:
path = "/home/jaime/PhD/LimberJack.jl/Notebooks/data/"
datas = [Data("DESgc", "DESgc", 2 , 2, path=path),
         Data("DESgc", "DESwl", 2 , 3, path=path)];
path_test= "/home/jaime/PhD/LimberJack.jl/test/data/"
datas_test1 = [Data("Dmygc", "Dmygc", 1 , 1, path=path_test),
         Data("Dmywl", "Dmywl", 2 , 2, path=path_test),
         Data("Dmygc", "Dmywl", 1 , 2, path=path_test)]
datas_test2 = [Data("Dmygc", "Dmygc", 1 , 1, path=path_test),
               Data("Dmygc", "Dmywl", 1 , 2, path=path_test),
               Data("Dmywl", "Dmywl", 2 , 2, path=path_test)]
Cls_metas = Cls_meta(datas, path=path);
Cls_metas_test1 = Cls_meta(datas_test1, path=path);
Cls_metas_test2 = Cls_meta(datas_test2, path=path);
cov_tot = Cls_metas.cov_tot;
data_vector = Cls_metas.data_vector;

In [3]:
cosmo = LimberJack.Cosmology(0.3, 0.05, 0.67, 0.96, 0.81,
                             tk_mode="EisHu", Pk_mode="Halofit");

In [4]:
theory = Theory(cosmo, Cls_metas; path=path);
theory_test1 = Theory(cosmo, Cls_metas_test1; path=path_test);
theory_test2 = Theory(cosmo, Cls_metas_test2; path=path_test);

In [5]:
theory_test1.Cls

3-element Vector{Float64}:
 1.404362970770941e-5
 1.7812471683040393e-9
 1.3480288330186787e-7

In [6]:
length(theory_test1.tracers) == length(theory_test2.tracers)

true

In [14]:
[typeof(tracer) for tracer in theory_test1.tracers]

2-element Vector{DataType}:
 NumberCountsTracer
 WeakLensingTracer

In [13]:
theory_test2.tracers[1]

NumberCountsTracer(400-element extrapolate(interpolate((::Vector{Float64},), ::Vector{Float64}, Gridded(Linear())), 0) with element type Float64:
 2.3866080925588832e-6
 7.17681612197578e-6
 1.2200164370949697e-5
 1.876347927105402e-5
 2.6186529691391524e-5
 3.492266205088506e-5
 4.423085245457592e-5
 5.339776531555639e-5
 6.35168401536602e-5
 7.511372770006339e-5
 9.109335234325195e-5
 0.0001113378948403124
 0.00013462698409213704
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0, 2.0, 0)

In [10]:
isequal(theory_test1.tracers, theory_test2.tracers)

false

In [8]:
@model function model(data_vector)
    Ωm ~ Uniform(0.2, 0.3)
    h ~ Uniform(0.6, 0.8)
    s8 ~ Uniform(0.7, 1.0)
    cosmology = LimberJack.Cosmology(Ωm, 0.05, h, 0.96, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    theory = Theory(cosmology, Cls_metas, path=path).Cls
    data_vector ~ MvNormal(theory, cov_tot)
end;

iterations = 500
step_size = 0.005
samples_per_step = 10
cores = 4

# Start sampling.
folname = string("DES_gc2gc2_gc2wl3_", "stpsz_", step_size, "_smpls_", samples_per_step)
if isdir(folname)
    println("Folder already exists")
    if isfile("chain.jls")
        println("Restarting from past chain")
        past_chain = read("chain.jls", Chains)
        new_chain = sample(model(data_vector), HMC(step_size, samples_per_step), iterations,
                           progress=true; save_state=true, resume_from=past_chain)
    end
else
    mkdir(folname)
    println("Created new folder")
    new_chain = sample(model(data_vector), HMC(step_size, samples_per_step),
                iterations, progress=true; save_state=true)
end

info = describe(new_chain)[1]
fname_info = string("info.csv")
CSV.write(joinpath(folname, fname_info), info)


fname_jls = string("chain.jls")
write(joinpath(folname, fname_jls), new_chain)
    
fname_csv = string("chain.csv")
CSV.write(joinpath(folname, fname_csv), new_chain)



Folder already exists


LoadError: UndefVarError: new_chain not defined